<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E8%8B%B1%E8%AF%AD%E5%AF%B9%E8%AF%9D%E6%8A%BD%E5%8F%96%E6%95%B0%E6%8D%AE%E7%9A%84%E6%95%B4%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/CardBuild/exp0122/first

0122english_0_to_10.txt		0122english_362_to_714.txt	 0123_eng_2nd_13406_to_13800.txt
0122english_10218_to_10570.txt	0122english_3882_to_4234.txt	 0123_eng_2nd_13800_to_14194.txt
0122english_10570_to_10922.txt	0122english_4234_to_4586.txt	 0123_eng_2nd_14194_to_14588.txt
0122english_1066_to_1418.txt	0122english_4586_to_4938.txt	 0123_eng_2nd_14588_to_14982.txt
0122english_10922_to_11274.txt	0122english_4938_to_5290.txt	 0123_eng_2nd_14982_to_15376.txt
0122english_10_to_362.txt	0122english_5290_to_5642.txt	 0123_eng_2nd_15376_to_15804.txt
0122english_11274_to_11626.txt	0122english_5642_to_5994.txt	 0123_eng_2nd_1586_to_1980.txt
0122english_11626_to_11978.txt	0122english_5994_to_6346.txt	 0123_eng_2nd_1980_to_2374.txt
0122english_11978_to_12330.txt	0122english_6346_to_6698.txt	 0123_eng_2nd_2374_to_2768.txt
0122english_12330_to_12682.txt	0122english_6698_to_7050.txt	 0123_eng_2nd_2768_to_3162.txt
0122english_12682_to_13034.txt	0122english_7050_to_7402.txt	 0123_eng_2nd_3162_to_3556.txt
01

In [5]:
import os
import json

# 指定文件夹路径
save_path = "/content/drive/MyDrive/CardBuild/exp0122/first"

all_files = []
# 遍历指定文件夹中的所有txt文件
for filename in os.listdir(save_path):
    if filename.endswith(".txt"):
        all_files.append(filename)



In [16]:
from tqdm import tqdm

all_datas = []

for filename in tqdm(all_files):

    # 构建完整的文件路径
    file_path = os.path.join(save_path, filename)

    error_count = 0

    # 打开并读取文件
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # 将每行解析为JSON
                data = json.loads(line)
                if "id" in data and "text" in data and "response" in data:
                    response_str = data["response"]
                    try:
                        response_json = json.loads(response_str)
                        data["response"] = response_json
                        all_datas.append(data)
                    except:
                        continue
                else:
                    continue


                # break
            except json.JSONDecodeError:
                # 如果解析JSON失败，跳过这一行
                continue

            # break

    # break


        # break

# 输出结果


100%|██████████| 92/92 [00:03<00:00, 25.74it/s]


In [17]:
print(len(all_datas))

33344


In [25]:
import random
n = len(all_datas)
sel_id = random.sample(range(n), 1)
print(all_datas[sel_id[0]]["text"][:500])

In [29]:
task_prompt = """Given an input paragraph, extract the dialogues within it, and output them in JSON format.

Let's think about it step by step:
- Summarize the input paragraph into bullet points and store it in the 'summary' field.
- Extract the content of each dialogue ('dialogue'), identify the speaker for each sentence ('said by'), and store these in 'conversations'."""

In [39]:
print(all_datas[20]["response"].keys())
print(all_datas[20]["response"]["conversations"])

print(all_datas[20]["response"]["conversations"][0].keys())

dict_keys(['summary', 'conversations'])
[{'num': 1, 'dialogue': 'I have read that there never was a system of oppression so bad that those who benefited by it did not recognize the moral sense so far as to make some excuse for themselves. Was the old system of property distribution, by which the few held the many in servitude through fear of starvation, an exception to this rule? Surely the rich could not have looked the poor in the face unless they had some excuse to offer, some color of reason to give for the cruel contrast between their conditions.', 'said by': 'Edith'}, {'num': 2, 'dialogue': 'Thanks for reminding us of that point. As you say, there never was a system so bad that it did not make an excuse for itself. It would not be strictly fair to the old system to dismiss it without considering the excuse made for it, although, on the other hand, it would really be kinder not to mention it, for it was an excuse that, far from excusing, furnished an additional ground of condemnat

In [54]:
def remove_ids(data):
    new_conv = []
    if "conversations" not in data:
        new_conv = []
    else:
        for conv in data["conversations"]:
            if "said_by" in conv:
                spk = conv["said_by"]
            elif "said by" in conv:
                spk = conv["said by"]
            else:
                spk = ""
            new_conv.append({
                "dialogue":conv["dialogue"],
                "said_by":spk
            })
    new_data = {
        "summary":data["summary"],
        "conversations":new_conv
    }
    return new_data

In [55]:
save_datas = []
for data in tqdm(all_datas):
    new_response = remove_ids(data["response"])
    json_str = json.dumps(new_response, ensure_ascii=False)
    conversations = [
        {"from":"human","value":data["text"]},
        {"from":"gpt","value":json_str}
    ]

    save_data = {
        "conversations":conversations,
        "system":task_prompt
    }
    save_datas.append(save_data)

100%|██████████| 33344/33344 [00:01<00:00, 31423.70it/s]


In [56]:
import os
import json
import random
def sample_and_save( save_datas, save_name ,sample_n=50, save_folder = "/content/drive/MyDrive/CardBuild/HaruhiZero", if_skip = True):
    file_name = os.path.join( save_folder, save_name + ".jsonl" )
    if os.path.exists( file_name ):
        if if_skip:
            print( "file ",file_name, "exists, skip" )
            return
    with open( file_name, "w", encoding="utf-8" ) as f:
        for data in save_datas:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

    sample_name = os.path.join( "/content/", save_name + "_sample.jsonl" )
    with open( sample_name, "w", encoding="utf-8" ) as f:
        random_50 = random. sample( save_datas, sample_n )
        for data in random_50:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

In [57]:
sample_and_save( save_datas, "english_dialogue_extract",if_skip = False )